In [1]:
import numpy as np

import os
import sys

from astropy.table import Table
from astropy import constants as const
from astropy import units as u
from astropy.io import fits
from astropy import coordinates as coords
from astropy.coordinates import SkyCoord, ICRS

import ResearchTools.LCtools as LCtools
import blosc
import pickle

import tqdm.autonotebook as tqdm

/var/folders/pq/0q6qw20n6xxb77g5kk6yy8480000gn/T/ipykernel_11550/1624798589.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import tqdm.autonotebook as tqdm


In [2]:
output_dir = "/Volumes/Helios/RESEARCH/TDSS_VARSTAR_LC_ANALYSIS_09-08-2021/Analysis_Results/"
datestr = '2021-09-08'

lc_dir0 = output_dir+datestr+"/ZTF/"
lc_dir_CSS = output_dir+datestr+"/CSS/"
lc_dir_ZTFg = output_dir+datestr+"/ZTF/g"
lc_dir_ZTFr = output_dir+datestr+"/ZTF/r"
lc_dir_ZTFi = output_dir+datestr+"/ZTF/i"

raw_lc_analysis_dir_ZTF = output_dir+datestr+"/RAW_LC_ANALYSIS/"+"ZTF/"
raw_LC_analysis_dir_CSS = output_dir+datestr+"/RAW_LC_ANALYSIS/"+"CSS/"
raw_LC_analysis_dir_ZTFg = output_dir+datestr+"/RAW_LC_ANALYSIS/"+"ZTF/g/"
raw_LC_analysis_dir_ZTFr = output_dir+datestr+"/RAW_LC_ANALYSIS/"+"ZTF/r/"
raw_LC_analysis_dir_ZTFi = output_dir+datestr+"/RAW_LC_ANALYSIS/"+"ZTF/i/"   


checkHarmonic = False
log10FAP = -5.0
logFAP_limit = log10FAP
polyfit_deg = 3

CSS_LC_dir = "/Users/benjaminroulston/Dropbox/Research/TDSS/Variable_Stars/HARD_COPY_ORGINAL_DATA/CSS_LCs/csvs/"
ZTF_LC_dir = "/Users/benjaminroulston/Dropbox/Research/TDSS/Variable_Stars/HARD_COPY_ORGINAL_DATA/ZTF/DATA/07-27-2021/"

ZTF_LC_data = Table.read("/Users/benjaminroulston/Dropbox/Research/TDSS/Variable_Stars/HARD_COPY_ORGINAL_DATA/ZTF/DATA/07-27-2021/TDSS_VarStar_ZTFDR6_gri_GroupID.fits")
TDSS_prop = Table.read("/Users/benjaminroulston/Dropbox/Research/TDSS/Variable_Stars/HARD_COPY_ORGINAL_DATA/PROGRAM_SAMPLE/2021-07-27/FINAL_FILES/TDSS_SES+PREV_DR16DR12griLT20_GaiaEDR3_Drake2014PerVar_CSSID_ZTFIDs_LCpointer_PyHammer_EqW.fits")

sample_size = len(TDSS_prop)

In [23]:
prop_id = 0
prop_id_last = 0
prop_col_names_prefix = ['CSS_', 'ZTF_g_', 'ZTF_r_']
prop_col_names = ['lc_id', 'P', 'logProb', 'Amp', 't0', 'Mt', 'a95', 'lc_skew',
                  'Chi2', 'brtcutoff', 'brt10per', 'fnt10per', 'fntcutoff', 'errmn', 'ferrmn',
                  'ngood', 'nrejects', 'nabove', 'nbelow', 'Tspan100', 'Tspan95', 'isAlias', 'time_whittened',
                  'VarStat', 'Con', 'm', 'b_lin', 'chi2_lin', 'a', 'b_quad', 'c', 'chi2_quad']

prop_col_names_full = [ii + jj for ii in prop_col_names_prefix for jj in prop_col_names]
prop_col_names_full.insert(0, 'ViCompleted')
prop_col_names_full.insert(0, 'dec')
prop_col_names_full.insert(0, 'ra')
# prop_col_names_full.append('EqW')

properties = np.zeros((len(TDSS_prop), len(prop_col_names_full))) * np.nan

properties = Table(properties, names=prop_col_names_full)
properties['ra'] = TDSS_prop['ra_GaiaEDR3']
properties['dec'] = TDSS_prop['dec_GaiaEDR3']
properties['ViCompleted'] = np.zeros(len(TDSS_prop))

properties

ra,dec,ViCompleted,CSS_lc_id,CSS_P,CSS_logProb,CSS_Amp,CSS_t0,CSS_Mt,CSS_a95,CSS_lc_skew,CSS_Chi2,CSS_brtcutoff,CSS_brt10per,CSS_fnt10per,CSS_fntcutoff,CSS_errmn,CSS_ferrmn,CSS_ngood,CSS_nrejects,CSS_nabove,CSS_nbelow,CSS_Tspan100,CSS_Tspan95,CSS_isAlias,CSS_time_whittened,CSS_VarStat,CSS_Con,CSS_m,CSS_b_lin,CSS_chi2_lin,CSS_a,CSS_b_quad,CSS_c,CSS_chi2_quad,ZTF_g_lc_id,ZTF_g_P,ZTF_g_logProb,ZTF_g_Amp,ZTF_g_t0,ZTF_g_Mt,ZTF_g_a95,ZTF_g_lc_skew,ZTF_g_Chi2,ZTF_g_brtcutoff,ZTF_g_brt10per,ZTF_g_fnt10per,ZTF_g_fntcutoff,ZTF_g_errmn,ZTF_g_ferrmn,ZTF_g_ngood,ZTF_g_nrejects,ZTF_g_nabove,ZTF_g_nbelow,ZTF_g_Tspan100,ZTF_g_Tspan95,ZTF_g_isAlias,ZTF_g_time_whittened,ZTF_g_VarStat,ZTF_g_Con,ZTF_g_m,ZTF_g_b_lin,ZTF_g_chi2_lin,ZTF_g_a,ZTF_g_b_quad,ZTF_g_c,ZTF_g_chi2_quad,ZTF_r_lc_id,ZTF_r_P,ZTF_r_logProb,ZTF_r_Amp,ZTF_r_t0,ZTF_r_Mt,ZTF_r_a95,ZTF_r_lc_skew,ZTF_r_Chi2,ZTF_r_brtcutoff,ZTF_r_brt10per,ZTF_r_fnt10per,ZTF_r_fntcutoff,ZTF_r_errmn,ZTF_r_ferrmn,ZTF_r_ngood,ZTF_r_nrejects,ZTF_r_nabove,ZTF_r_nbelow,ZTF_r_Tspan100,ZTF_r_Tspan95,ZTF_r_isAlias,ZTF_r_time_whittened,ZTF_r_VarStat,ZTF_r_Con,ZTF_r_m,ZTF_r_b_lin,ZTF_r_chi2_lin,ZTF_r_a,ZTF_r_b_quad,ZTF_r_c,ZTF_r_chi2_quad
deg,deg,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
0.01307073795,-2.43008296623,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0.01470748805,20.15233374844,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0.02193203895,1.5599789031,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0.02716434631,26.08194970618,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0.03463319489,0.52495551063,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan

In [39]:
prop_col_names_prefix
prop_col_names

for prop_id, ROW in enumerate(tqdm.tqdm(TDSS_prop)):
    CSS_pickle_filename = raw_LC_analysis_dir_CSS + f"{prop_id}_CSS_{ROW['CSSID']}.dat"
    ZTFg_pickle_filename = raw_lc_analysis_dir_ZTF + f"g/" + f"{prop_id}_ZTFg_{ROW['ZTF_GroupID']}.xz"
    ZTFr_pickle_filename = raw_lc_analysis_dir_ZTF + f"r/" + f"{prop_id}_ZTFr_{ROW['ZTF_GroupID']}.xz"
    
    if os.path.isfile(CSS_pickle_filename):
        with open(CSS_pickle_filename, 'rb') as f:
            compressed_pickle = f.read()

        depressed_pickle = blosc.decompress(compressed_pickle)
        props = pickle.loads(depressed_pickle) 
        
        properties[prop_id]['CSS_lc_id'] = ROW['CSSID']
        for col_name_end in prop_col_names[1:]:
            col_name = prop_col_names_prefix[0] + col_name_end
            properties[prop_id][col_name] = props[col_name_end]
        
    if os.path.isfile(ZTFg_pickle_filename):
        with open(ZTFg_pickle_filename, 'rb') as f:
            compressed_pickle = f.read()

        depressed_pickle = blosc.decompress(compressed_pickle)
        props = pickle.loads(depressed_pickle) 
        
        properties[prop_id]['ZTF_g_lc_id'] = ROW['ZTF_GroupID']
        for col_name_end in prop_col_names[1:]:
            col_name = prop_col_names_prefix[1] + col_name_end
            properties[prop_id][col_name] = props[col_name_end]
        
    if os.path.isfile(ZTFr_pickle_filename):
        with open(ZTFr_pickle_filename, 'rb') as f:
            compressed_pickle = f.read()

        depressed_pickle = blosc.decompress(compressed_pickle)
        props = pickle.loads(depressed_pickle) 
        
        properties[prop_id]['ZTF_r_lc_id'] = ROW['ZTF_GroupID']
        for col_name_end in prop_col_names[1:]:
            col_name = prop_col_names_prefix[2] + col_name_end
            properties[prop_id][col_name] = props[col_name_end] 
    

  0%|          | 0/25121 [00:00<?, ?it/s]

In [45]:
properties.write("TDSS_VarStar_LCprops_09-08-2021.fits", format='fits', overwrite=True)

properties

ra,dec,ViCompleted,CSS_lc_id,CSS_P,CSS_logProb,CSS_Amp,CSS_t0,CSS_Mt,CSS_a95,CSS_lc_skew,CSS_Chi2,CSS_brtcutoff,CSS_brt10per,CSS_fnt10per,CSS_fntcutoff,CSS_errmn,CSS_ferrmn,CSS_ngood,CSS_nrejects,CSS_nabove,CSS_nbelow,CSS_Tspan100,CSS_Tspan95,CSS_isAlias,CSS_time_whittened,CSS_VarStat,CSS_Con,CSS_m,CSS_b_lin,CSS_chi2_lin,CSS_a,CSS_b_quad,CSS_c,CSS_chi2_quad,ZTF_g_lc_id,ZTF_g_P,ZTF_g_logProb,ZTF_g_Amp,ZTF_g_t0,ZTF_g_Mt,ZTF_g_a95,ZTF_g_lc_skew,ZTF_g_Chi2,ZTF_g_brtcutoff,ZTF_g_brt10per,ZTF_g_fnt10per,ZTF_g_fntcutoff,ZTF_g_errmn,ZTF_g_ferrmn,ZTF_g_ngood,ZTF_g_nrejects,ZTF_g_nabove,ZTF_g_nbelow,ZTF_g_Tspan100,ZTF_g_Tspan95,ZTF_g_isAlias,ZTF_g_time_whittened,ZTF_g_VarStat,ZTF_g_Con,ZTF_g_m,ZTF_g_b_lin,ZTF_g_chi2_lin,ZTF_g_a,ZTF_g_b_quad,ZTF_g_c,ZTF_g_chi2_quad,ZTF_r_lc_id,ZTF_r_P,ZTF_r_logProb,ZTF_r_Amp,ZTF_r_t0,ZTF_r_Mt,ZTF_r_a95,ZTF_r_lc_skew,ZTF_r_Chi2,ZTF_r_brtcutoff,ZTF_r_brt10per,ZTF_r_fnt10per,ZTF_r_fntcutoff,ZTF_r_errmn,ZTF_r_ferrmn,ZTF_r_ngood,ZTF_r_nrejects,ZTF_r_nabove,ZTF_r_nbelow,ZTF_r_Tspan100,ZTF_r_Tspan95,ZTF_r_isAlias,ZTF_r_time_whittened,ZTF_r_VarStat,ZTF_r_Con,ZTF_r_m,ZTF_r_b_lin,ZTF_r_chi2_lin,ZTF_r_a,ZTF_r_b_quad,ZTF_r_c,ZTF_r_chi2_quad
deg,deg,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
0.01307073795,-2.43008296623,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.0,0.24681607266265176,-17.153094192298454,0.628701611311854,59230.0703425807,0.23473131656646729,1.1401158332824686,-0.7536162331215583,352.4385865066225,16.62615207210183,16.76460208892822,17.932926750183107,18.01073008030653,0.024543970823287964,0.024705590680241585,152.0,3.0,56.0,86.0,949.7024421000024,870.1212951400012,1.0,1.0,15.445170402526855,0.013333333333333334,-7.7873148537469e-05,17.421733046151143,309.1939170698177,2.616695979318502e-07,-0.0003287839050195231,17.457354088840965,308.47662195086065,0.0,0.24683069389044662,-15.521757591339796,0.47931650838365414,59230.061263378215,0.28639334440231323,0.8604145050048828,-0.7790990413953,219.3022518382353,16.702128566801548,16.776626205444337,17.65270233154297,17.737619623541832,0.021040966734290123,0.02110491320490837,171.0,2.0,65.0,91.0,955.6273727000007,860.6887824000005,1.0,1.0,12.583793640136719,0.0,0.00012133781230780936,17.209341260668747,199.31445297827065,2.605051375878538e-07,-0.0001277742708714937,17.24659666098282,198.3365552974925
0.01470748805,20.15233374844,0.0,1121001009174.0,0.14090699011396557,-2.47780669100987,0.07093727543014694,57390.031253170404,0.6460357812174616,0.2620813999999996,1.132069770998009,1.4664662293831858,17.139910538645466,17.2905252,17.568765,17.894912790370487,0.07047702605795554,0.0696473676985877,299.0,6.0,11.0,14.0,3741.865743483402,3645.753828747678,0.0,0.0,1.308166017288768,0.0,-3.6699571005916473e-06,17.41905008020355,1.4249483524692808,3.3454698690383447e-09,-1.6516710894834636e-05,17.42672156915522,1.421424410248135,1.0,0.1383745966380508,-1.4157158454814665,0.015981277150700635,59233.13241341902,0.6041442155838013,0.08016281127929759,0.15757650225323516,0.9495281420255962,17.78492620959878,17.84242477416992,17.92433891296387,17.99490600